In [57]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score


import warnings
warnings.filterwarnings('ignore')

In [58]:
df_train = pd.read_csv("./Credit score/train.csv", low_memory=False)
df_train.head()

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good


1. Для выполнения задания вы можете самостоятельно создать Python-ноутбук, а в LMS прикладывать лишь требуемые результаты.

Будем работать с датасетом для предсказания кредитного дефолта.

Можно скачать датасет

Целевой признак Credit_Score

Выясните, клиенты с каким значением признака Payment_Behaviour более склонны к просрочкам по кредиту (Credit_Score=Poor)?

In [59]:
poor_behaviour = df_train[df_train["Credit_Score"] == "Poor"]
poor_behaviour["Payment_Behaviour"].value_counts()

Payment_Behaviour
Low_spent_Small_value_payments      9125
High_spent_Medium_value_payments    4439
Low_spent_Medium_value_payments     4211
High_spent_Small_value_payments     3088
High_spent_Large_value_payments     2979
Low_spent_Large_value_payments      2931
!@9#%8                              2225
Name: count, dtype: int64

2. Для выполнения задания вы можете самостоятельно создать Python-ноутбук, а в LMS прикладывать лишь требуемые результаты.

Продолжайте работать с датасетом для предсказания кредитного дефолта.

В данных есть заведомо неинформативные столбцы, поэтому их нужно удалить.

Укажите в ответе названия этих столбцов через пробел в одной строке.

Формат ответа:  Признак1 Признак2
В ответе может быть любое количество признаков.

In [60]:
df_train.describe(include='all')

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
count,100000,100000,100000,90015,100000,100000,100000,100000,84998.000000,100000.000000,...,100000,100000,100000.000000,90970,100000,100000.000000,95521,100000,98800,100000
unique,100000,12500,8,10139,1788,12501,16,18940,NaN,NaN,...,4,13178,NaN,404,3,NaN,91049,7,98792,3
top,0x1602,CUS_0xd40,January,Langep,38,#F%$D@*&8,_______,36585.12,NaN,NaN,...,Standard,1360.45,NaN,15 Years and 11 Months,Yes,NaN,__10000__,Low_spent_Small_value_payments,__-333333333333333333333333333__,Standard
freq,1,8,12500,44,2833,5572,7062,16,NaN,NaN,...,36479,24,NaN,446,52326,NaN,4305,25513,9,53174
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4194.170850,17.091280,...,NaN,NaN,32.285173,NaN,NaN,1403.118217,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3183.686167,117.404834,...,NaN,NaN,5.116875,NaN,NaN,8306.041270,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,303.645417,-1.000000,...,NaN,NaN,20.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1625.568229,3.000000,...,NaN,NaN,28.052567,NaN,NaN,30.306660,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3093.745000,6.000000,...,NaN,NaN,32.305784,NaN,NaN,69.249473,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5957.448333,7.000000,...,NaN,NaN,36.496663,NaN,NaN,161.224249,NaN,NaN,NaN,NaN


In [61]:
"ID Customer_ID Name SSN"

'ID Customer_ID Name SSN'

3/ Для выполнения задания вы можете самостоятельно создать Python-ноутбук, а в LMS прикладывать лишь требуемые результаты.

Продолжайте работать с датасетом для предсказания кредитного дефолта.

Удалите явные пропуски в данных. Изучите признаки в датасете, убедитесь, что типы данных соответствуют значениям в признаке.

Если есть несостыковки по типам данных, то удалите те строки, где тип значения в признаке не соответствует типу признака (для удобства можете эти значения переводить в NaN).

Оставьте в выборке только численные признаки.

Укажите в ответе, сколько осталось объектов в выборке.

In [62]:
col_drop = ["ID", "Customer_ID", "Name", "SSN"]
df = df_train.drop(columns=col_drop)
df

,Month,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,January,23,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",...,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,February,23,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",...,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,March,-500,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",...,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,April,23,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",...,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,May,23,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",...,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,April,25,Mechanic,39628.99,3359.415833,4,6,7,2,"Auto Loan, and Student Loan",...,_,502.38,34.663572,31 Years and 6 Months,No,35.104023,60.97133255718485,High_spent_Large_value_payments,479.86622816574095,Poor
99996,May,25,Mechanic,39628.99,3359.415833,4,6,7,2,"Auto Loan, and Student Loan",...,_,502.38,40.565631,31 Years and 7 Months,No,35.104023,54.18595028760385,High_spent_Medium_value_payments,496.651610435322,Poor
99997,June,25,Mechanic,39628.99,3359.415833,4,6,5729,2,"Auto Loan, and Student Loan",...,Good,502.38,41.255522,31 Years and 8 Months,No,35.104023,24.02847744864441,High_spent_Large_value_payments,516.8090832742814,Poor
99998,July,25,Mechanic,39628.99,3359.415833,4,6,7,2,"Auto Loan, and Student Loan",...,Good,502.38,33.638208,31 Years and 9 Months,No,35.104023,251.67258219721603,Low_spent_Large_value_payments,319.1649785257098,Standard


In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 24 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Month                     100000 non-null  object 
 1   Age                       100000 non-null  object 
 2   Occupation                100000 non-null  object 
 3   Annual_Income             100000 non-null  object 
 4   Monthly_Inhand_Salary     84998 non-null   float64
 5   Num_Bank_Accounts         100000 non-null  int64  
 6   Num_Credit_Card           100000 non-null  int64  
 7   Interest_Rate             100000 non-null  int64  
 8   Num_of_Loan               100000 non-null  object 
 9   Type_of_Loan              88592 non-null   object 
 10  Delay_from_due_date       100000 non-null  int64  
 11  Num_of_Delayed_Payment    92998 non-null   object 
 12  Changed_Credit_Limit      100000 non-null  object 
 13  Num_Credit_Inquiries      98035 non-null   fl

In [64]:
df['Age'] = pd.to_numeric(df['Age'], errors='coerce')

df['Annual_Income'] = pd.to_numeric(df['Annual_Income'], errors='coerce')

df['Num_of_Loan'] = pd.to_numeric(df['Num_of_Loan'], errors='coerce')

df['Num_of_Delayed_Payment'] = pd.to_numeric(
    df['Num_of_Delayed_Payment'], errors='coerce')

df['Changed_Credit_Limit'] = pd.to_numeric(
    df['Changed_Credit_Limit'], errors='coerce')

df['Outstanding_Debt'] = pd.to_numeric(
    df['Outstanding_Debt'], errors='coerce')

df['Amount_invested_monthly'] = pd.to_numeric(
    df['Amount_invested_monthly'], errors='coerce')

df['Monthly_Balance'] = pd.to_numeric(
    df['Monthly_Balance'], errors='coerce')

df = df.dropna()
df.shape

(44568, 24)

In [65]:
# Оставляем только численные признаки
columns_to_stay = df.dtypes[df.dtypes != 'object'].index.to_list()
y = df["Credit_Score"]
X = df[columns_to_stay]
print(X.shape)
X.head()

(44568, 16)


,Age,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Outstanding_Debt,Credit_Utilization_Ratio,Total_EMI_per_month,Amount_invested_monthly,Monthly_Balance
0,23.0,19114.12,1824.843333,3,4,3,4.0,3,7.0,11.27,4.0,809.98,26.822620,49.574949,80.415295,312.494089
9,28.0,34847.84,3037.986667,2,4,6,1.0,7,1.0,7.42,2.0,605.03,38.550848,18.816215,40.391238,484.591214
12,28.0,34847.84,3037.986667,2,4,6,1.0,3,1.0,6.42,2.0,605.03,34.977895,18.816215,130.115420,444.867032
13,28.0,34847.84,3037.986667,2,4,6,1.0,3,0.0,5.42,2.0,605.03,33.381010,18.816215,43.477190,481.505262
15,28.0,34847.84,3037.986667,2,4,6,1.0,3,4.0,5.42,2.0,605.03,32.933856,18.816215,218.904344,356.078109


Задание 4.

Продолжайте работать с датасетом для предсказания кредитного дефолта.

Сделайте разбиение на обучение и тест с соотношением 80/20, со стратификацией по целевому признаку и random_state=1.

Выполните обучение модели SVC с параметрами по умолчанию.

Выведите опорные векторы (для этого используйте атрибут .support_vectors_).

В ответе укажите количество векторов.

In [66]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1, stratify=y)

In [67]:
# Обучение модели SVC с параметрами по умолчанию
model = SVC()
model.fit(X_train, y_train)

# Получение опорных векторов
support_vectors = model.support_vectors_

# Вывод количества опорных векторов
print(f"Количество опорных векторов: {len(model.support_vectors_)}")

Количество опорных векторов: 31337


Задание 5

Чтобы модель быстрее обучалась возьмите выборку меньшей размерности - на тренировочной выборке оставьте 10000 случайных объектов, а на тесте - 2000.

Получайте индексы объектов через np.random.choice с replace=False, зафиксируйте случайность для трейна через np.random.seed(1), а для теста - np.random.seed(2).

Обучите еще раз модели SVC на параметрах по умолчанию, но перед этим сделайте масштабирование данных через стандартизацию, можете воспользоваться методом make_pipeline

Сделайте предсказания моделью на тестовой выборке и посчитайте метрику recall для каждого класса. Для удобства можете воспользоваться функцией classification_report.

Укажите в ответе значения метрик, округлив их до второго знака после точки.

Формат ответа:  0.70

In [68]:
# Установка семян для случайности
np.random.seed(1)

# Выбор случайных объектов для тренировочной выборки
train_indices = np.random.choice(X.index, size=10000, replace=False)
X_train = X.loc[train_indices]
y_train = y.loc[train_indices]

# Установка семян для тестовой выборки
np.random.seed(2)

# Выбор случайных объектов для тестовой выборки
test_indices = np.random.choice(X.index.difference(
    train_indices), size=2000, replace=False)
X_test = X.loc[test_indices]
y_test = y.loc[test_indices]

# Создание и обучение модели SVC с масштабированием
pipeline = make_pipeline(StandardScaler(), SVC())
pipeline.fit(X_train, y_train)

# Предсказания на тестовой выборке
y_pred = pipeline.predict(X_test)

# Оценка модели с помощью recall
report = classification_report(y_test, y_pred, output_dict=True)
recall_values = {key: round(value['recall'], 2) for key, value in report.items(
) if key not in ['accuracy', 'macro avg', 'weighted avg']}

# Вывод значений метрик
print(recall_values)

{'Good': 0.04, 'Poor': 0.54, 'Standard': 0.86}


6. Для выполнения задания вы можете самостоятельно создать Python-ноутбук, а в LMS прикладывать лишь требуемые результаты.

Продолжайте работать с датасетом для предсказания кредитного дефолта.

У модели SVC изменяйте ядра: ['linear', 'rbf', 'poly'], другие параметры оставьте по умолчанию.

Используйте масштабированные данные.

Укажите в ответе ядро, при котором получилась самая высокая метрика на тесте recall с макро усреднением.

In [69]:
model_linear = Pipeline([
    ('scaler', StandardScaler()),
    ('svc', SVC(kernel="linear"))
])


model_linear.fit(X_train, y_train)

y_pred_lin = model_linear.predict(X_test)

report_lin = classification_report(y_test, y_pred_lin)
print(report_lin)

              precision    recall  f1-score   support

        Good       0.00      0.00      0.00       320
        Poor       0.62      0.48      0.54       629
    Standard       0.58      0.83      0.68      1051

    accuracy                           0.59      2000
   macro avg       0.40      0.44      0.41      2000
weighted avg       0.50      0.59      0.53      2000



In [70]:
model_rbf = Pipeline([
    ('scaler', StandardScaler()),
    ('svc', SVC(kernel="rbf"))
])

model_rbf.fit(X_train, y_train)

y_pred_rbf = model_rbf.predict(X_test)

report_rbf = classification_report(y_test, y_pred_rbf)
print(report_rbf)

              precision    recall  f1-score   support

        Good       0.55      0.04      0.07       320
        Poor       0.69      0.54      0.60       629
    Standard       0.60      0.86      0.71      1051

    accuracy                           0.62      2000
   macro avg       0.61      0.48      0.46      2000
weighted avg       0.62      0.62      0.57      2000



In [71]:
model_poly = Pipeline([
    ('scaler', StandardScaler()),
    ('svc', SVC(kernel="poly"))
])


model_poly.fit(X_train, y_train)

y_pred_poly = model_poly.predict(X_test)

report_poly = classification_report(y_test, y_pred_poly)
print(report_poly)

              precision    recall  f1-score   support

        Good       0.35      0.03      0.06       320
        Poor       0.71      0.27      0.39       629
    Standard       0.56      0.92      0.70      1051

    accuracy                           0.57      2000
   macro avg       0.54      0.41      0.38      2000
weighted avg       0.57      0.57      0.50      2000



7. Для выполнения задания вы можете самостоятельно создать Python-ноутбук, а в LMS прикладывать лишь требуемые результаты.

Продолжайте работать с датасетом для предсказания кредитного дефолта.

У модели SVC возьмите самое лучшее ядро с предыдущего задания, другие параметры оставьте по умолчанию.

Подберите самое лучшее значение параметра C из списка [0.001, 0.01, 0.1, 1, 10, 100].

Используйте масштабированные данные.

Визуализируйте изменение метрики recall_macro на тренировочной и тестовой выборках от параметра C.

Укажите в ответе значение C, при котором получилась самая высокая метрика на тесте recall с макро усреднением.

In [72]:
C = [0.001, 0.01, 0.1, 1, 10, 100]
recall_score = dict()
for i in C:
    model_rbf_c = Pipeline([
        ('scaler', StandardScaler()),
        ('svc', SVC(kernel="rbf", C=i))
    ])
    model_rbf_c.fit(X_train, y_train)
    y_pred_rbf_c = model_rbf_c.predict(X_test)
    report_rbf_c = classification_report(
        y_test, y_pred_rbf_c, output_dict=True)
    rec = report_rbf_c['macro avg']["recall"]

    recall_score[i] = rec
    
recall_score

{0.001: 0.3333333333333333,
 0.01: 0.4124675971656438,
 0.1: 0.4465245454779232,
 1: 0.47706273254280757,
 10: 0.5005456452002458,
 100: 0.5282339081259577}

8. Задание 8.

Возьмите самые лучшие гиперпараметры модели с предыдущего задания, сделайте предсказания на тестовой выборке и посчитайте метрику ROC-AUC, параметр multi_class возьмите такой же (ovr и ovo), как и в модели SVC по умолчанию.

Для удобства можно перевести строковые целевые значения в порядковые числа через LabelEncoder.

Используйте масштабированные данные.

Обратите внимание, что по умолчанию модель SVC не умеет предсказывать вероятности.

В ответе укажите метрику RCO-AUC с округлением до второго знака после точки.

In [73]:
labels = LabelEncoder()
y_train_labels = labels.fit_transform(y_train)
y_test_labels = labels.transform(y_test)

In [74]:
svm_model = SVC(kernel='rbf', C=100, probability=True)
pipe = Pipeline([
    ('scale', StandardScaler()),
    ("svc", svm_model),
])

pipe.fit(X_train, y_train_labels)
y_proba = pipe.predict_proba(X_test)

roc_auc = roc_auc_score(y_test_labels, y_proba, multi_class='ovr')
print(f"ROC-AUC: {roc_auc:.2f}")

ROC-AUC: 0.77


Задание №9.

Давайте убедимся, что подобранные гиперпараметры модели точно самые лучшие, для этого сделайте:

перебор ядер ['rbf', 'poly'] (линейное убрали)

перебор значения C из списка [0.001, 0.01, 0.1, 1, 10, 100]

другие параметры оставьте по умолчанию.

Используйте масштабированные данные.

Считайте качество модели по кросс-валидации на 3 фолда (cross_val_score) на метрике recall с макро усреднением только на тренировочной выборке.

Укажите в ответе лучшие гиперпараметры и метрику по кросс-валидации, округлив её до второго знака после точки.

Формат ввода: 0.40

In [75]:
param_grid = {
    'svc__kernel': ['rbf', 'poly'],  # Перебор ядер
    'svc__C': [0.001, 0.01, 0.1, 1, 10, 100]  # Перебор значений C
}

pipe = Pipeline([
    ('scale', StandardScaler()),
    ("svc", SVC()),
])


grid_search = GridSearchCV(pipe, param_grid, cv=3,
                           scoring='recall_macro', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Лучшие параметры и метрика
best_params = grid_search.best_params_
best_score = grid_search.best_score_

# Вывод результатов
print(f"Лучшие параметры: {best_params}")
print(f"Метрика recall (macro): {best_score:.2f}")

Лучшие параметры: {'svc__C': 100, 'svc__kernel': 'rbf'}
Метрика recall (macro): 0.53
